In [1]:
#Random Forest algorithm
#replicate findings from Walters et al 2014

In [2]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

import biom
import sklearn
import pandas as pd

In [3]:
#Amish dataset
text_file = "/home/blooh/ersp/merged_bmi_mapping_final__original_study_amish_Fraser__.txt"
biom_file = "/home/blooh/ersp/filtered_otu_table__original_study_amish_Fraser__.biom"

In [4]:
m = pd.read_csv(text_file, sep="\t", index_col=0)

In [5]:
m.columns

Index([u'BarcodeSequence', u'LinkerPrimerSequence', u'BMI',
       u'bmi_group_binned', u'bmi_group_coded', u'original_study',
       u'combined_study_bmi_group', u'PCR_PRIMERS', u'TARGET_SUBFRAGMENT',
       u'AGE', u'ELEVATION', u'LONGITUDE', u'COUNTRY', u'SEQUENCING_METH',
       u'SAMPLE_CENTER', u'Description_duplicate', u'ReversePrimer',
       u'COLLECTION_DATE', u'SEX', u'FAMILY_RELATIONSHIP_GG', u'STUDY_CENTER',
       u'EXPERIMENT_CENTER', u'bmi_group_amish', u'RUN_CENTER', u'LATITUDE',
       u'Description'],
      dtype='object')

In [6]:
m.index


Int64Index([  1,   6,  16,  18,  32,  35,  53,  54,  60,  64,
            ...
            471, 473, 475, 476, 481, 482, 488, 595, 598, 599],
           dtype='int64', name=u'#SampleID', length=325)

In [7]:
b= biom.load_table(biom_file)


In [8]:
#select only samples that fit LEAN OR OBESE
i = m[m.bmi_group_coded != "Overweight"].index
print(i[0])
i_2 = m[m.bmi_group_coded != "Overweight"]

i_2.loc[i_2.bmi_group_binned == 3, 'bmi_group_binned'] = 0
#print(i_2)

1


/home/blooh/.local/lib/python2.7/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
writer = pd.ExcelWriter('biom_original_1.xlsx', engine='xlsxwriter')



In [10]:
p = pd.DataFrame(b.matrix_data.T.todense().astype(int), index=b.ids(axis="sample"), columns=b.ids(axis="observation"))
print(p)

     304309  193008  190299  191393  193061  186596  174924  296166  177792  \
274       1       0       0       0       0       0       0       0      45   
35        0       1       0       0       0       0       2       0       0   
360       0       1       0       0       0       0       0       0       0   
347       0       1       2       0       0       0       0       0       0   
349       0       2       1       0       0       0       0       0       0   
329       0       0       2       0       0       0       0       0      10   
336       0       0       3       0       0       0       1       0       0   
132       0       0       2       0       0       0       0       0      29   
171       0       0       1       0       0       0       0       0       0   
148       0       0       1       0       0       0       0       0       1   
443       0       0       1       0       0       1       0       0       2   
435       0       0       1       0       0       0 

In [11]:
# Convert the dataframe to an XlsxWriter Excel object.
p.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [12]:
# load sorted file
sorted_m = pd.read_csv("/Users/shwetakinger/biom_original_1.csv", index_col=0)
print(sorted_m)

IOError: File /Users/shwetakinger/biom_original_1.csv does not exist

In [ ]:
#sorted_m.index.get_loc(6)
#sorted_m.iloc[[4]]

data = pd.DataFrame([])
for n in range(0, 222):
    #print(n)
    num = sorted_m.index.get_loc(i[n])
    data = data.append(sorted_m.iloc[[num]])
    
print(data)


In [ ]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier


In [ ]:
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
iter_range = range(1, 6)
# empty list to store scores
rForest_scores = []
average_accuracy = 0.0;

for i in iter_range:
    classifier = RandomForestClassifier(n_estimators=1000)
    #cross-validation score
    #scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_coded, cv=10, scoring='accuracy')
    predicted = cross_val_predict(classifier, data, i_2.bmi_group_binned, cv=10)
    #cross-validation AUC score
    #scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_binned, cv=10, scoring='roc_auc')
    #rForest_scores.append(scores.mean())
    #average_accuracy+=scores.mean()
    print(confusion_matrix(i_2.bmi_group_binned, predicted))

#print(rForest_scores)
#print(average_accuracy/(5.0))

In [ ]:
from sklearn.cross_validation import cross_val_score
iter_range = range(1, 6)
# empty list to store scores
rForest_scores = []
average_accuracy = 0.0;

for i in iter_range:
    classifier = RandomForestClassifier(n_estimators=1000)
    #cross-validation score
    #scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_coded, cv=10, scoring='accuracy')
    #cross-validation AUC score
    scores = cross_val_score(classifier, data, i_2.bmi_group_binned, cv=10, scoring='roc_auc')
    rForest_scores.append(scores.mean())
    average_accuracy+=scores.mean()

print(rForest_scores)
print(average_accuracy/(5.0))